<a href="https://colab.research.google.com/github/Maria-Moskaleva/Analysis_of_Weather/blob/main/Analysis_of_Weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving FAPE_2013_2023.csv to FAPE_2013_2023 (2).csv


In [ ]:
# считывание csv файла с разделителем ',' колонка valid как дата
metar_PE = pd.read_csv('FAPE_2013_2023.csv', sep=",", parse_dates=[1])
metar_PE.head()

,station,valid,metar
0,FAPE,2013-01-01 00:00:00,FAPE 010000Z 25003KT 9999 -RA SCT013 BKN020 OV...
1,FAPE,2013-01-01 01:00:00,FAPE 010100Z 24005KT 9999 -RA FEW015 BKN020 OV...
2,FAPE,2013-01-01 02:00:00,FAPE 010200Z 20004KT 170V230 8000 -RA BKN005 O...
3,FAPE,2013-01-01 03:00:00,FAPE 010300Z 22006KT 9999 -RA BKN005 BKN015 16...
4,FAPE,2013-01-01 04:00:00,FAPE 010400Z 23005KT 9999 BKN040 16/14 Q1017 N...


In [ ]:
# отберем данные со временем от 6 до 18. по-хорошему притянуть таблицу с восходами и закатами, так как зимой день короче
metar_PE['hour'] = metar_PE.valid.dt.hour
metar_PE = metar_PE.query("hour <= 18 and hour >= 6")
metar_PE.shape

(50030, 4)

In [ ]:
#часть кодов Metar начинается не с аэропорта, проблемы со сплитованием. COR - скорректированная автоматическая сводка
starts_fape = metar_PE.metar.apply(lambda x: x.startswith('FAPE'))
metar_PE.loc[:, 'starts_fape'] = starts_fape
metar_PE[starts_fape == False].starts_fape.count()

77

In [ ]:
#удалим COR из кода METAR
correct_metar = metar_PE.metar.apply(lambda x: x.strip('COR '))
metar_PE.loc[:, 'metar'] = correct_metar

In [ ]:
#проверка
starts_fape = metar_PE.metar.apply(lambda x: x.startswith('FAPE'))
metar_PE.loc[:, 'starts_fape'] = starts_fape
metar_PE[starts_fape == False].starts_fape.count() #теперь все коды METAR начинаются с кода аэропорта

0

In [ ]:
#проверим коды METAR на наличие AUTO перед ветром
wind_bag_auto = metar_PE.metar.str.contains('AUTO')
metar_PE.loc[:, 'wind_bag_auto'] = wind_bag_auto
metar_PE[wind_bag_auto == True].metar.count()

514

In [ ]:
#удалим AUTO из кода METAR
without_auto = metar_PE.metar.apply(lambda x: x.replace(' AUTO ', ' '))
metar_PE.loc[:, 'metar'] = without_auto
metar_PE[wind_bag_auto == True].head() #проверка

,station,valid,metar,hour,starts_fape,wind_bag_auto
58947,FAPE,2020-03-27 14:00:00,FAPE 271400Z 14009KT 9999 ///////// 20/13 Q1019,14,True,True
58948,FAPE,2020-03-27 15:00:00,FAPE 271500Z 13008KT 9999 ///////// 20/13 Q1019,15,True,True
58949,FAPE,2020-03-27 16:00:00,FAPE 271600Z 10006KT 9999 ///////// 18/12 Q1019,16,True,True
58950,FAPE,2020-03-27 17:00:00,FAPE 271700Z 10004KT 9999 ///////// 16/12 Q1019,17,True,True
58951,FAPE,2020-03-27 18:00:00,FAPE 271800Z 07006KT 9999 ///////// 17/14 Q1020,18,True,True


In [ ]:
#проверим коды METAR на наличие VRB
wind_bag_VRB = metar_PE.metar.str.contains('VRB')
metar_PE.loc[:, 'wind_bag_VRB'] = wind_bag_VRB
metar_PE[wind_bag_VRB == True].head() #что делать??

,station,valid,metar,hour,starts_fape,wind_bag_auto,metar_1,wind_bag_VRB
102,FAPE,2013-01-05 08:00:00,FAPE 050800Z VRB06KT CAVOK 23/12 Q1016 NOSIG,8,True,False,FAPE 050800Z VRB06KT CAVOK 23/12 Q1016 NOSIG,True
251,FAPE,2013-01-11 16:00:00,FAPE 111600Z VRB03KT 9999 FEW036 21/12 Q1018 N...,16,True,False,FAPE 111600Z VRB03KT 9999 FEW036 21/12 Q1018 N...,True
264,FAPE,2013-01-12 06:00:00,FAPE 120600Z VRB02KT 9999 FEW013 BKN025 18/16 ...,6,True,False,FAPE 120600Z VRB02KT 9999 FEW013 BKN025 18/16 ...,True
384,FAPE,2013-01-17 08:00:00,FAPE 170800Z VRB03KT CAVOK 27/20 Q1011 NOSIG,8,True,False,FAPE 170800Z VRB03KT CAVOK 27/20 Q1011 NOSIG,True
386,FAPE,2013-01-17 10:00:00,FAPE 171000Z VRB05KT CAVOK 29/20 Q1012 NOSIG,10,True,False,FAPE 171000Z VRB05KT CAVOK 29/20 Q1012 NOSIG,True


In [ ]:
# отдельный столбец с ветром
metar_PE['wind'] = metar_PE.metar.apply(lambda x: x.split(' ')[2])
metar_PE.shape
#!!! проверить на пустые значения wind

(50030, 7)

In [ ]:
# отберем нужные столбцы
metar_PE = metar_PE.loc[:, ['valid', 'metar', 'wind']]
metar_PE.head()

,valid,metar,wind
6,2013-01-01 06:00:00,FAPE 010600Z 25009KT 9999 SCT048 19/15 Q1017 N...,25009KT
7,2013-01-01 07:00:00,FAPE 010700Z 23010KT 9999 BKN048 20/15 Q1017 N...,23010KT
8,2013-01-01 08:00:00,FAPE 010800Z 23011KT 9999 FEW040 21/14 Q1017 N...,23011KT
9,2013-01-01 09:00:00,FAPE 010900Z 23013KT 9999 SCT022 21/15 Q1018 N...,23013KT
10,2013-01-01 10:00:00,FAPE 011000Z 23013KT 200V260 9999 SCT027 20/14...,23013KT


In [ ]:
#проблемы для выделения ветра: VRB, //. Поищем такие данные
wind_bag = metar_PE.wind.str.contains('//')
metar_PE.loc[:, 'wind_bag'] = wind_bag
metar_PE[wind_bag == True] # можем удалить?

,valid,metar,wind,wind_bag
8572,2014-01-16 07:00:00,FAPE 160700Z /////KT 9999 BKN028 23/17 Q1017 N...,/////KT,True
8573,2014-01-16 08:00:00,FAPE 160800Z /////KT 9999 SCT028 24/18 Q1017 N...,/////KT,True
8574,2014-01-16 09:00:00,FAPE 160900Z /////KT 9999 FEW018 26/20 Q1016 N...,/////KT,True
8575,2014-01-16 10:00:00,FAPE 161000Z /////KT 9999 FEW018 26/19 Q1016 N...,/////KT,True
8576,2014-01-16 11:00:00,FAPE 161100Z /////KT 9999 FEW015 26/19 Q1016 N...,/////KT,True
...,...,...,...,...
86971,2023-07-15 06:00:00,FAPE 150600Z /////KT CAVOK ///// Q//// NOSIG,/////KT,True
86972,2023-07-15 07:00:00,FAPE 150700Z /////KT CAVOK ///// Q//// NOSIG,/////KT,True
86973,2023-07-15 08:00:00,FAPE 150800Z /////KT CAVOK 20/08 Q//// NOSIG,/////KT,True
89754,2023-11-12 06:00:00,FAPE 120600Z /////KT 9999 BKN038 OVC048 ///// ...,/////KT,True


In [ ]:
metar_PE['wind_direction'] = metar_PE.wind.apply(lambda x: x[:3])
metar_PE['wind_speed'] = metar_PE.wind.apply(lambda x: x[3:-2])
metar_PE.groupby('wind_direction').agg({'wind_speed': 'count'})

,wind_speed
wind_direction,
///,384
000,175
008,1
010,511
020,586
030,811
040,873
050,1019
060,1345


In [ ]:
metar_PE.sort_values('wind_direction')

,valid,metar,wind,wind_bag,wind_direction,wind_speed
60792,2020-06-14 17:00:00,FAPE 141700Z 26010KT 9999 FEW069/// BKN240///...,,False,,
60746,2020-06-12 18:00:00,FAPE 121800Z 28011KT 9999 SCT240/// 13/05 Q10...,,False,,
60745,2020-06-12 17:00:00,FAPE 121700Z 27016KT 9999 SCT240/// 14/05 Q10...,,False,,
60744,2020-06-12 16:00:00,FAPE 121600Z 26019KT 9999 SCT250/// 14/06 Q10...,,False,,
60743,2020-06-12 15:00:00,FAPE 121500Z 26015KT 9999 SCT250/// 14/07 Q10...,,False,,
...,...,...,...,...,...,...
75687,2022-03-20 06:00:00,FAPE 200600Z VRB01KT 9999 FEW002 SCT045 20/20 ...,VRB01KT,False,VRB,01
45885,2018-08-22 08:00:00,FAPE 220800Z VRB03KT 9999 SCT040 15/10 Q1022 N...,VRB03KT,False,VRB,03
45929,2018-08-24 06:00:00,FAPE 240600Z VRB02KT CAVOK 11/09 Q1023 NOSIG,VRB02KT,False,VRB,02
45953,2018-08-25 07:00:00,FAPE 250700Z VRB02KT CAVOK 18/11 Q1007 NOSIG,VRB02KT,False,VRB,02


In [ ]:
metar_PE = metar_PE.query("wind_speed < '20'") #ветер не больше 20 узлов
metar_PE = metar_PE.query("wind_direction !='VRB' and wind_direction != '///'")

In [ ]:
#изменим тип данных на числовой
#выкинем симводы из WD
metar_PE_wind = metar_PE.iloc[:, [0, 1, 2, 4, 5, 6]]

In [ ]:
#выкинули символы в wind_speed, поменяла тип данных
metar_PE_wind['bed_speed'] = metar_PE_wind.wind_speed.str.contains('G')
#metar_PE_wind['bed_speed'] = metar_PE_wind.wind_speed.str.contains('//')
metar_PE_wind = metar_PE_wind.query("bed_speed ==False")
metar_PE_wind.wind_speed.value_counts()

10    355
08    339
09    329
11    314
07    298
12    278
06    272
13    236
05    221
15    208
14    204
04    200
16    178
17    165
03    140
18    137
19    106
00     16
02     11
01      1
Name: wind_speed, dtype: int64

In [ ]:
metar_PE_wind['wind_speed'] = metar_PE_wind.wind_speed.astype(float)
metar_PE_wind['wind_direction'] = metar_PE_wind.wind_direction.astype(float)

In [ ]:
#def f(x):
 # if good_data_new.wind_direction>170 and good_data_new.wind_direction<350:
  #  cross_wind=good_data_new.wind_speed*sin(abs(80-good_data_new.wind_direction))
  #else:
   # cross_wind=good_data_new.wind_speed*sin(abs(260-good_data_new.wind_direction))


In [ ]:
metar_PE_wind.dtypes

valid             datetime64[ns]
metar                     object
hour                       int64
wind                      object
wind_direction           float64
wind_speed               float64
bed_speed                   bool
dtype: object

In [ ]:
import math
def f(x, y):
  if x['wind_direction']>170 and x['wind_direction']<350:
    return y['wind_speed']*np.sin(80-x)
  else:
    return y['wind_speed']*np.sin(260-x)

In [ ]:
import numpy as np
metar_PE_wind['new_column'] = abs(metar_PE_wind.wind_speed*np.sin(80-metar_PE_wind.wind_direction)) # нужен модуль или нет?

In [ ]:
def calculate_cross_wind(row):
    if row['wind_direction'] > 170 and row['wind_direction']<350:
        return abs(row['wind_speed']*np.sin(80 - row['wind_direction']))
    else:
        return abs(row['wind_speed']*np.sin(260 - row['wind_direction']))

# Применяем функцию к DataFrame для создания новой колонки "cross_wind"
metar_PE_wind['cross_wind'] = metar_PE_wind.apply(calculate_cross_wind, axis=1)

In [ ]:
metar_PE_wind = metar_PE_wind.query("cross_wind < 8") #боковой ветер меньше 8 узлов

In [ ]:
metar_PE_wind['day'] = metar_PE_wind.valid.dt.date

In [ ]:
metar_PE_wind['month'] = metar_PE_wind.valid.dt.month

In [ ]:
metar_PE_wind.groupby('month').agg({'metar': 'count'}).sort_values('metar', ascending=False).head(5)
# май, июнь, июль - лучшие месяца по ветру

,metar
month,
5,295
6,246
7,245
1,243
4,241


In [ ]:
metar_PE_wind['CAVOK'] = metar_PE_wind.metar.str.contains('CAVOK')
metar_PE_without_cavok = metar_PE_wind.query("CAVOK == False")
#отберем метар без CAVOK, чтобы отобрать по облачности и видимости

metar_PE_without_cavok = metar_PE_without_cavok.loc[:, ['valid', 'metar', 'month']] #отберем только нужные колонки
metar_PE_without_cavok.head()

,valid,metar,month
7,2023-01-01 07:00:00,FAPE 010700Z 05008KT 020V080 9999 FEW008 22/17...,1
8,2023-01-01 08:00:00,FAPE 010800Z 04010KT 020V080 9999 FEW008 22/17...,1
9,2023-01-01 09:00:00,FAPE 010900Z 06009KT 030V110 9999 FEW020 23/17...,1
78,2023-03-01 13:15:00,FAPE 031315Z 08009KT 050V110 9999 FEW060TCU 31...,3
79,2023-03-01 14:00:00,FAPE 031400Z 10008KT 040V150 9999 FEW060TCU 30...,3


In [ ]:
metar_PE_without_cavok['visibility'] = metar_PE_without_cavok.metar.apply(lambda x: x.split(' ')[4])
metar_PE_without_cavok.head()

,valid,metar,month,wind,visibility
7,2023-01-01 07:00:00,FAPE 010700Z 05008KT 020V080 9999 FEW008 22/17...,1,9999,9999
8,2023-01-01 08:00:00,FAPE 010800Z 04010KT 020V080 9999 FEW008 22/17...,1,9999,9999
9,2023-01-01 09:00:00,FAPE 010900Z 06009KT 030V110 9999 FEW020 23/17...,1,9999,9999
78,2023-03-01 13:15:00,FAPE 031315Z 08009KT 050V110 9999 FEW060TCU 31...,3,9999,9999
79,2023-03-01 14:00:00,FAPE 031400Z 10008KT 040V150 9999 FEW060TCU 30...,3,9999,9999


In [ ]:
#посчитаем длину кода METAR для того, чтобы понимать какие данные в него входят
metar_PE['len_metar'] = metar_PE.metar.apply(lambda x: len(x.split(' ')))
metar_PE.groupby('len_metar').agg({'valid': 'count'})

,valid
len_metar,
6,1
7,1199
8,1331
9,783
10,376
11,239
12,169
13,110
14,41


In [ ]:
metar_PE.query("len_metar == 14")

,valid,metar,hour,starts_fape,wind,wind_direction,wind_speed,len_metar
137,2023-05-01 07:15:00,FAPE 050715Z 24015KT 7000 -DZRA SCT007 BKN016 ...,7,True,24015KT,240,15,14
1449,2023-02-17 10:13:00,FAPE 171013Z 12008KT 090V160 9999 TS FEW020 BK...,10,True,12008KT,120,08,14
1452,2023-02-17 11:00:00,FAPE 171100Z 21011KT 170V240 9999 -TSRA FEW014...,11,True,21011KT,210,11,14
1454,2023-02-17 11:54:00,FAPE 171154Z 17007KT 120V200 9999 -TSRA SCT015...,11,True,17007KT,170,07,14
1455,2023-02-17 12:00:00,FAPE 171200Z 17007KT 120V200 9999 -TSRA SCT015...,12,True,17007KT,170,07,14
1456,2023-02-17 13:00:00,FAPE 171300Z 08015KT 8000 -TSRA FEW010 FEW040C...,13,True,08015KT,080,15,14
1483,2023-02-18 09:00:00,FAPE 180900Z 22009KT 3500 -RA BR SCT009 BKN025...,9,True,22009KT,220,09,14
1752,2023-02-27 13:37:00,FAPE 271337Z 19008KT 160V230 2800 DZRA BKN006 ...,13,True,19008KT,190,08,14
1753,2023-02-27 14:00:00,FAPE 271400Z 19007KT 170V230 2800 -RA BR BKN00...,14,True,19007KT,190,07,14
3287,2023-04-18 07:00:00,FAPE 180700Z 25004KT 200V300 0500 R26/0650D FG...,7,True,25004KT,250,04,14
